In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U peft
!pip install -q -U trl
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U transformers
!pip install -q -U wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 75.3 MB/s eta 0:00:00


In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

In [ ]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
model_id = "microsoft/Phi-3-mini-4k-instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ["HF_TOKEN"])
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"":0},
    token = os.environ["HF_TOKEN"]
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
text = "quote: Imagination is more,"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

quote: Imagination is more, than the ability to create, it is the ability to see the invisible, to dream the impossible, to believe the unbelievable. - Albert Einstein

In this quote, Albert Einstein is emphasizing the power of imagination. He


In [ ]:
text = "quote: Imagination is more,"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

quote: Imagination is more, than the ability to create, it is the ability to see the invisible, to dream the impossible, to believe the unbelievable. - Albert Einstein

In this quote, Albert Einstein is emphasizing the power of imagination. He suggests that imagination is not just about the ability to create something new or tangible, but it'ieves the invisible, dream the impossible, and believe the unbelievable. This means that imagination allows us to envision things that are not currently present or real, to think beyond the constraints of reality, and to hold onto the belief that what seems unbelievable can become possible.

Einstein's words highlight the transformative power of imagination. It's the force that drives innovation, discovery, and progress. Imagination enables us to think creatively, to solve problems in novel ways, and to envision a future that goes beyond the present limitations. It's the spark that ignites the human


In [ ]:
os.environ["WANDB_DISABLED"] = "false"

In [ ]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [ ]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
data['train']['quote']

Column(['“Be yourself; everyone else is already taken.”', "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”", "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”", '“So many books, so little time.”', '“A room without books is like a body without a soul.”'])

In [ ]:
def formatting_func(example):
  text = f"Quote: {example['quotw'][0]}\nAuthor: {example['author'][0]}"
  return [text]

In [ ]:
data['train']

Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 2508
})

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = data["train"],
    args = transformers.TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 2,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 1,
        output_dir = "outputs",
        optim = "paged_adamw_8bit"
  ),
    peft_config = lora_config,
    formatting_func = formatting_func,
)

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:687: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a formatting function. Therefore `formatting_func` will be ignored. Either remove the `formatting_func` or pass a dataset that is not already processed.
  warnings.warn(


Truncating train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: agrawalpraveen9698 (agrawalpraveen9698-indian-institute-of-information-techn) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,3.231800
2,2.115300
3,2.505700
4,3.042700
5,2.583100
6,2.990400
7,2.863300
8,2.149500
9,3.126400
10,2.187900


TrainOutput(global_step=100, training_loss=2.278921412229538, metrics={'train_runtime': 269.0174, 'train_samples_per_second': 1.487, 'train_steps_per_second': 0.372, 'total_flos': 379151310428160.0, 'train_loss': 2.278921412229538})

In [ ]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_Phi-3-mini")
tokenizer.save_pretrained("fine_tuned_Phi-3-mini")

('fine_tuned_Phi-3-mini/tokenizer_config.json',
 'fine_tuned_Phi-3-mini/special_tokens_map.json',
 'fine_tuned_Phi-3-mini/chat_template.jinja',
 'fine_tuned_Phi-3-mini/tokenizer.model',
 'fine_tuned_Phi-3-mini/added_tokens.json',
 'fine_tuned_Phi-3-mini/tokenizer.json')